<a href="https://colab.research.google.com/github/digitalblue/aaie-model-lab/blob/prompting%2Fqwen-basic-prompting-notebook/Qwen_AAIE_Basic_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Qwen basic prompting notebook
This notebook for setup of inference with Qwen models using Huggingface framework. The Huggingface frameworks allows for working with many other open models such as Mistral, Llama, TinyLlama, Deepseek and more.

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
    pipeline
)
import torch

In [ ]:
# log into hugging face , add you token via environment variables or Secrets pane on Colab
hf_token = userdata.get('HF_TOKEN')
login(hf_token)


### Assesment Feedback Generation
Setup of mock prompts for chat template

In [ ]:
system_prompt = """
You are a helpful and respectful educational assessment assistant that provides feedback on submitted work.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Provide assessment feedback and a rating for the assessment.
Ensure your assessment is accurate, do not guess or make assumptions.
If you something is unclear seek clarification from the student.
"""

In [ ]:
rubric_prompt = """

### **Rubric for Basic Data Structures Written Assignment**
**Course Name**: Introduction to Data Structures
**Assignment**: Comparing Arrays and Linked Lists
**Total Points**: 100

| **Criteria**               | **Excellent (A) 90-100%** | **Good (B) 80-89%** | **Satisfactory (C) 70-79%** | **Needs Improvement (D/F) ≤69%** | **Points Earned** |
|----------------------------|---------------------------|---------------------|-----------------------------|----------------------------------|------------------|
| **Definition & Core Concepts** | - Clearly defines the data structure(s) (e.g., "A stack is LIFO").<br>- Accurately describes key operations (e.g., push/pop for stacks). | - Minor omissions or vague definitions.<br>- Operations are listed but not deeply explained. | - Partial definitions or errors in operations.<br>- Confuses similar structures (e.g., queue vs. stack). | - Incorrect or missing definitions.<br>- No clear understanding of operations. | /20 |
| **Correctness & Examples** | - Provides accurate, well-explained examples (e.g., "How a queue manages a print spooler").<br>- No logical/technical errors. | - Examples are correct but lack depth.<br>- Minor errors in explanations. | - Examples are overly simplistic or contain errors.<br>- Confuses edge cases (e.g., empty stack pop). | - Incorrect or missing examples.<br>- Fundamental misunderstandings. | /25 |
| **Basic Time Complexity** | - Correctly identifies time/space complexity for operations (e.g., "O(1) for stack push").<br>- Explains *why* (e.g., "No shifting needed in linked lists"). | - Complexity is mostly correct but lacks justification.<br>- Misses edge cases (e.g., dynamic array resizing). | - Errors in complexity (e.g., "O(n) for stack push").<br>- No explanation. | - No complexity analysis or entirely wrong. | /20 |
| **Comparison (if required)** | - Clearly contrasts structures (e.g., "Arrays allow random access; linked lists don’t").<br>- Uses tables/diagrams effectively. | - Comparisons are present but lack depth.<br>- Minimal use of visuals. | - Superficial comparisons (e.g., "Arrays are faster").<br>- No diagrams. | - No meaningful comparison. | /15 |
| **Clarity & Organization** | - Logical flow (intro → explanation → examples → conclusion).<br>- No grammar/spelling errors. | - Mostly clear but some awkward phrasing.<br>- Few typos. | - Hard to follow in sections.<br>- Several writing errors. | - Disorganized; errors impede understanding. | /10 |
| **Code (if applicable)** | - Clean, working code with comments.<br>- Matches theoretical explanations. | - Code works but lacks comments.<br>- Minor inefficiencies. | - Code is messy or partially incorrect.<br>- No comments. | - Code doesn’t run or is irrelevant. | /10 |

"""

In [ ]:
assessment_prompt = """

### **Key Adjustments for "Basic" Level:**
1. **Simplified Complexity**: Focus on **O(1)/O(n)** (no need for amortized/recursive analysis).
2. **Emphasis on Clarity**: Reward clear definitions and examples over advanced optimizations.
3. **Visual Aids**: Encourage diagrams (e.g., "Draw a linked list insertion") for better understanding.

### **Grading Notes:**
- **A**: No errors; explanations are beginner-friendly and insightful.
- **B**: Minor errors but demonstrates core understanding.
- **C**: Needs improvement in accuracy or clarity.
- **D/F**: Fundamental gaps in knowledge.

### **Key Differences Summary**
| Aspect       | Poor (D/F)          | Average (C)         | Excellent (A)       |
|-------------|--------------------|--------------------|--------------------|
| Accuracy    | Major errors       | Minor errors       | Perfect            |
| Depth       | Superficial        | Basic              | Detailed           |
| Examples    | None               | Generic            | Real-world         |
| Clarity     | Confusing          | Readable           | Flawless           |

"""

In [ ]:
submission_1_prompt = """
**Assignment:** *"Compare and contrast arrays and linked lists. Explain their operations, time complexities, and use cases."*

**Student Submission:**

Arrays and linked lists are ways to store data. Arrays are fast because they are in one place in memory. Linked lists are slow and nobody uses them.

Arrays can hold things like names. Linked lists have nodes and pointers, which is confusing. To get something from an array, you just type `array[0]`. For linked lists, you have to do a loop, which is bad.

Arrays are better because they are simple. Linked lists are useless because they take too much memory. Computers are fast anyway, so speed doesn’t matter.

**Operations:**
- Arrays: `insert`, `delete`, `search` (all fast).
- Linked lists: `insert`, `delete`, `search` (all slow).

**Conclusion:** Always use arrays.
"""

submission_1_feedback_prompt = """
### **1. Poor Submission (Grade: D/F)**

**Why it’s poor:**
❌ Incorrect time complexities (claims all array ops are O(1)).
❌ No real examples or diagrams.
❌ Misunderstands linked lists entirely.
❌ No clear structure or justification.

"""

In [ ]:
submission_2_prompt = """
**Assignment:** *"Compare and contrast arrays and linked lists. Explain their operations, time complexities, and use cases."*

**Student Submission:**

**Introduction**
Arrays and linked lists store data but work differently.

**Arrays**
- Contiguous memory blocks.
- Fixed size (unless dynamic).
- Operations:
  - Access: O(1) (fast).
  - Insert/delete at end: O(1).
  - Insert/delete in middle: O(n) (slow).

**Linked Lists**
- Nodes with pointers to next element.
- Dynamic size (can grow easily).
- Operations:
  - Access: O(n) (slow).
  - Insert/delete at head: O(1).
  - Insert/delete in middle: O(n).

**Comparison**
| Feature       | Array          | Linked List     |
|--------------|---------------|----------------|
| Access       | O(1)          | O(n)           |
| Insert (mid) | O(n)          | O(n)           |
| Memory       | Fixed         | Dynamic        |

**Use Cases**
- Arrays: Good for frequent access (e.g., storing grades).
- Linked lists: Good for frequent inserts (e.g., playlist).

**Conclusion**
Arrays are faster for access; linked lists are flexible.
"""

submission_2_feedback_prompt = """
### **2. Average Submission (Grade: C)**

**Why it’s average:**
✔ Correct basics but lacks depth.
✔ Minor errors (e.g., doesn’t mention amortized O(1) for dynamic arrays).
✔ No diagrams or code examples.
✔ Basic comparison but no real-world context.

"""

In [ ]:
submission_3_prompt = """
**Assignment:** *"Compare and contrast arrays and linked lists. Explain their operations, time complexities, and use cases."*

**Student Submission:**

**Introduction**
Arrays and linked lists are fundamental data structures with distinct trade-offs in memory, access, and modification.

**Arrays**
- **Structure:** Contiguous memory.
- **Operations:**
  - Random access: O(1) (e.g., `arr[3]`).
  - Insert/delete at end: O(1) (amortized for dynamic arrays).
  - Insert/delete in middle: O(n) (requires shifting).
- **Use Case:** Ideal for CPU caches (fast access) or fixed-size datasets.

**Linked Lists**
- **Structure:** Nodes with `data` + `next` pointer.
- **Operations:**
  - Access head/tail: O(1).
  - Random access: O(n) (must traverse).
  - Insert/delete at head: O(1).
  - Insert/delete in middle: O(n) (but no shifting!).
- **Use Case:** Browser history (frequent inserts/deletes).

**In-Depth Comparison**
| Feature       | Array                          | Linked List                  |
|--------------|-------------------------------|------------------------------|
| Memory       | Wastes space if resized       | Extra overhead per node      |
| Access       | O(1) (ideal for binary search)| O(n) (inefficient for lookup)|
| Insert (mid) | O(n) (shifting)               | O(n) (but no shifting!)      |

**Real-World Example**
- **Array:** Image processing (pixels stored contiguously).
- **Linked List:** Undo/redo functionality (efficient inserts).

**Conclusion**
Arrays excel at fast access; linked lists win for dynamic data. Choose based on application needs.
"""

submission_3_feedback_prompt = """
### **3. Excellent Submission (Grade: A)**

**Why it’s excellent:**
✔ Precise definitions and examples.
✔ Correct time complexities (including amortized analysis).
✔ Real-world use cases and clear comparisons.
✔ Professional structure and flawless clarity.

"""

Function to format system and user prompts as prompt template

In [ ]:
def get_prompt_template(system_prompt, user_pronpt):
  messages = [
    {
        "role": "system",
        "content": system_prompt,
    },
    {"role": "user", "content": user_pronpt},
  ]
  return messages

In [ ]:
message = get_prompt_template(system_prompt, submission_1_prompt)

In [ ]:
print(message)

[{'role': 'system', 'content': '\nYou are a helpful and respectful educational assessment assistant that provides feedback on submitted work. \nAlways answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. \nPlease ensure that your responses are socially unbiased and positive in nature.\nEnsure your assessment is accurate, do not guess or make assumptions.\nIf you something is unclear seek clarification from the student.\n'}, {'role': 'user', 'content': '\n**Assignment:** *"Compare and contrast arrays and linked lists. Explain their operations, time complexities, and use cases."*  \n\n**Student Submission:**  \n\nArrays and linked lists are ways to store data. Arrays are fast because they are in one place in memory. Linked lists are slow and nobody uses them.  \n\nArrays can hold things like names. Linked lists have nodes and pointers, which is confusing. To get something from an a

## Qwen 2.5
Inference on Qwen 2.5 0.5B model. This is a small model and can run on CPU.

https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct

In [ ]:
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct")
pipe(message)

Device set to use cpu


[{'generated_text': [{'role': 'system',
    'content': '\nYou are a helpful and respectful educational assessment assistant that provides feedback on submitted work. \nAlways answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. \nPlease ensure that your responses are socially unbiased and positive in nature.\nEnsure your assessment is accurate, do not guess or make assumptions.\nIf you something is unclear seek clarification from the student.\n'},
   {'role': 'user',
    'content': '\n**Assignment:** *"Compare and contrast arrays and linked lists. Explain their operations, time complexities, and use cases."*  \n\n**Student Submission:**  \n\nArrays and linked lists are ways to store data. Arrays are fast because they are in one place in memory. Linked lists are slow and nobody uses them.  \n\nArrays can hold things like names. Linked lists have nodes and pointers, which is confus

## Qwen 3
Inference on Qwen 3.0 0.6B model. This is newer version and includes improved reasoning.

https://huggingface.co/Qwen/Qwen3-0.6B

In [ ]:
pipe = pipeline("text-generation", model="Qwen/Qwen3-0.6B")
pipe(message)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Device set to use cpu


[{'generated_text': [{'role': 'system',
    'content': '\nYou are a helpful and respectful educational assessment assistant that provides feedback on submitted work. \nAlways answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. \nPlease ensure that your responses are socially unbiased and positive in nature.\nEnsure your assessment is accurate, do not guess or make assumptions.\nIf you something is unclear seek clarification from the student.\n'},
   {'role': 'user',
    'content': '\n**Assignment:** *"Compare and contrast arrays and linked lists. Explain their operations, time complexities, and use cases."*  \n\n**Student Submission:**  \n\nArrays and linked lists are ways to store data. Arrays are fast because they are in one place in memory. Linked lists are slow and nobody uses them.  \n\nArrays can hold things like names. Linked lists have nodes and pointers, which is confus